In [6]:
import  time
import numpy as np
import pandas as pd
from pyecharts.charts import Bar3D

In [21]:
sheet_name=['2015','2016','2017','2018','会员等级']
sheet_datas=[pd.read_excel('data/sales.xlsx',sheet_name=i) for i in sheet_name]

In [22]:
pd.read_excel('data/sales.xlsx',sheet_name=sheet_name) 

{'2015':               会员ID         订单号       提交日期    订单金额
 0      15278002468  3000304681 2015-01-01   499.0
 1      39236378972  3000305791 2015-01-01  2588.0
 2      38722039578  3000641787 2015-01-01   498.0
 3      11049640063  3000798913 2015-01-01  1572.0
 4      35038752292  3000821546 2015-01-01    10.1
 ...            ...         ...        ...     ...
 30769  39368100847  4281994827 2015-12-31   828.0
 30770       409757  4282010457 2015-12-31   199.0
 30771  38380526114  4282017675 2015-12-31   208.0
 30772     28074988  4282019440 2015-12-31    89.0
 30773  39460363230  4282025309 2015-12-31   719.0
 
 [30774 rows x 4 columns],
 '2016':               会员ID         订单号       提交日期     订单金额
 0      39288120141  4282025766 2016-01-01    76.00
 1      39293812118  4282037929 2016-01-01  7599.00
 2      27596340905  4282038740 2016-01-01   802.00
 3      15111475509  4282043819 2016-01-01    65.00
 4      38896594001  4282051044 2016-01-01    95.00
 ...            ...         ...

In [14]:
for eache_name,eache_data in zip(sheet_name,sheet_datas):
    print("============="+eache_name+"==========")
    print(eache_data.head(4))
    print(eache_data.describe())
    print(eache_data.info())
    print(eache_data.isnull().sum())

=============2015==========
          会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
               会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30774 entries, 0 to 30773
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   会员ID    30774 non-null  int64         
 1   订单号     30774 non-null  int64         
 2   提交日期    3077

In [28]:
#数据简单清洗
for ind,eache_data in enumerate(sheet_datas[:-1]):
    sheet_datas[ind]=eache_data.dropna()
    sheet_datas[ind]=eache_data[eache_data['订单金额']>1]
    sheet_datas[ind]['max_year_data']=eache_data['提交日期'].max()

In [29]:
sheet_datas[0].head()

,会员ID,订单号,提交日期,订单金额,max_year_data
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31


In [33]:
data_merge=pd.concat(sheet_datas[:-1],axis=0)
data_merge['year']=data_merge['提交日期'].dt.year

,会员ID,订单号,提交日期,订单金额,max_year_data,year
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,2015


In [38]:
data_merge['date_interva']=data_merge['max_year_data']-data_merge['提交日期']
data_merge['date_interva']=data_merge['date_interva'].dt.days
data_merge.head()

,会员ID,订单号,提交日期,订单金额,max_year_data,year,date_interva
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,2015,364
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,2015,364
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,2015,364
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,2015,364
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,2015,364


In [41]:
rfm_gb=data_merge.groupby(['year','会员ID'],as_index=False).agg({'date_interva':'min','提交日期':'count','订单金额':'sum'})
rfm_gb.columns=['year','会员ID','r','f','m']
rfm_gb.head()

,year,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0


In [43]:
desc_pd=rfm_gb.iloc[:,2:].describe().T
desc_pd

,count,mean,std,min,25%,50%,75%,max
r,148591.0,165.524043,101.988472,0.0,79.0,156.0,255.0,365.0
f,148591.0,1.365002,2.626953,1.0,1.0,1.0,1.0,130.0
m,148591.0,1323.741329,3753.906883,1.5,69.0,189.0,1199.0,206251.8


In [45]:
r_bins=[-1,79,255,365]
f_bins=[0,2,5,130]
m_bins=[0,69,1199,206252]

In [74]:
rfm_gb['r_source']=pd.cut(rfm_gb['r'],r_bins,labels=[i for i in range(len(r_bins)-1,0,-1)])
rfm_gb['f_source']=pd.cut(rfm_gb['f'],f_bins,labels=[i for i in range(1,len(f_bins),1)])
rfm_gb['m_source']=pd.cut(rfm_gb['m'],m_bins,labels=[i for i in range(1,len(m_bins),1)])

In [68]:
rfm_gb.sort_values('f_source',ascending=False)

,year,会员ID,r,f,m,r_source,f_source,m_source,rfm_group
118172,2018,34871189725,167,17,9841.5,2,3,3,b'2'b'3'b'3'
116028,2018,32988584511,217,16,10375.7,2,3,3,b'2'b'3'b'3'
72608,2017,29333869243,138,16,8582.6,2,3,3,b'2'b'3'b'3'
72603,2017,29333308526,21,53,42595.0,3,3,3,b'3'b'3'b'3'
37586,2016,26065007457,282,19,53417.1,1,3,3,b'1'b'3'b'3'
...,...,...,...,...,...,...,...,...,...
49844,2016,39057960315,120,1,99.0,2,1,2,b'2'b'1'b'2'
49843,2016,39057951433,278,1,59.0,1,1,1,b'1'b'1'b'1'
49842,2016,39057918173,232,1,159.0,2,1,2,b'2'b'1'b'2'
49841,2016,39057903502,163,2,456.0,2,1,2,b'2'b'1'b'2'


In [75]:
rfm_gb['r_source']=rfm_gb['r_source'].astype(np.str_)
rfm_gb['f_source']=rfm_gb['f_source'].astype(np.str_)
rfm_gb['m_source']=rfm_gb['m_source'].astype(np.str_)

In [77]:
rfm_gb['rfm_group']=rfm_gb['r_source'].str.cat(rfm_gb['f_source']).str.cat(rfm_gb['m_source'])
rfm_gb.head()

,year,会员ID,r,f,m,r_source,f_source,m_source,rfm_group
0,2015,267,197,2,105.0,2,1,2,212
1,2015,282,251,1,29.7,2,1,1,211
2,2015,283,340,1,5398.0,1,1,3,113
3,2015,343,300,1,118.0,1,1,2,112
4,2015,525,37,3,213.0,3,2,2,322


In [86]:
display_data = rfm_gb.groupby(['rfm_group','year'],as_index=False)['会员ID'].count()
display_data.columns = ['rfm_group','year','number']
display_data['rfm_group'] = display_data['rfm_group'].astype(int)
display_data.head()

,rfm_group,year,number
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811


In [88]:
from pyecharts.charts import Bar3D
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
range_max = int(display_data['number'].max())
c = (
    Bar3D()#设置了一个3D柱形图对象
    .add(
        "",#标题
        [d.tolist() for d in display_data.values],#数据
        xaxis3d_opts=opts.Axis3DOpts( type_="category",name='分组名称'),#x轴数据类型，名称
        yaxis3d_opts=opts.Axis3DOpts( type_="category",name='年份'),#y轴数据类型，名称
        zaxis3d_opts=opts.Axis3DOpts(type_="value",name='会员数量'),#z轴数据类型，名称
        
    )
    .set_global_opts(#设置颜色，及不同取值对应的颜色
        visualmap_opts=opts.VisualMapOpts(max_=range_max,range_color=range_color),
        title_opts=opts.TitleOpts(title="RFM分组结果") ) #设置标题 
    
)
c.render('网页.html') 

'D:\\wym\\work\\my\\pythonProject2\\网页.html'